In [1]:
from datamodule import Sec_Data_Module
from model import Sec_Classifier
from config import config
import os
import pytorch_lightning as pl

/home/jh6/patent_env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
!export CUDA_VISIBLE_DEVICES=2

In [3]:
config

{'model_name': 'bert-base-uncased',
 'n_labels': 3,
 'batch_size': 64,
 'lr': 1.5e-05,
 'warmup': 0.2,
 'weight_decay': 0.001,
 'n_epochs': 20}

In [4]:
model_list = ['xlm-roberta-base', 'bert-base-uncased', 'jackaduma/SecRoBERTa']
for model_name in model_list:
    config['model_name'] = model_name
    epoch_num = config['n_epochs']

    print(f"{epoch_num}epoch train started!")

    model_name = config['model_name'].split('/')[-1]
    data_path = f'../{model_name}_dataset/'
    train_path = data_path + 'train.pkl'
    val_path = data_path + 'val.pkl'

    data_module = Sec_Data_Module(train_path, val_path, batch_size=config['batch_size'], model_name=config['model_name'], max_token_len=512)
    data_module.setup()
    config['train_size'] = len(data_module.train_dataloader())
    model = Sec_Classifier(config)

    trainer = pl.Trainer(max_epochs=config['n_epochs'], num_sanity_val_steps=-1, accelerator="gpu", devices=1)
    # trainer = pl.Trainer(max_epochs=config['n_epochs'], num_sanity_val_steps=-1, accelerator="gpu", devices=-1, strategy="ddp_find_unused_parameters_true")
    # trainer = pl.Trainer(max_epochs=config['n_epochs'], accelerator="gpu", devices=4, strategy="dp")
    trainer.fit(model, data_module)

    if not os.path.exists(f'{model_name}_ckpt'):
        os.mkdir(f'{model_name}_ckpt')

    model_save_dir = f'./{model_name}_ckpt/'

    # trainer.save_checkpoint(model_save_dir, f'{model_name}_{epoch_num}Model.ckpt')

    # predictions = trainer.predict(model, data_module)
    # torch.save(model.state_dict(), model_save_dir+f'{model_name}_{epoch_num}Model.pth')

    # with open(pred_path+f'{sample_num}SamplePredictions.pickle', 'wb') as fw:
    #     pickle.dump(predictions, fw)

    print(f"{epoch_num}epoch train completed!")

20epoch train started!


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: usually670. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jh6/patent_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/gen

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 79.19 GiB total capacity; 33.89 GiB already allocated; 57.62 MiB free; 34.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 